# HashiCorp Vault Demo for KMIP Secrets Engine and MySQL Enterprise TDE

The OASIS Key Management Interoperability Protocol (KMIP) standard is a widely adopted protocol for handling cryptographic workloads and secrets management for enterprise infrastructure such as databases, network storage, and virtual/physical servers.  When an organization has services and applications that need to perform cryptographic operations (e.g.: transparent database encryption (TDE), full disk encryption, etc.), it often delegates the key management task to an external provider via KMIP protocol. 

This demo demonstrates HashiCorp Vault's KMIP Secrets Engine to allow Vault to function as a KMIP server for clients that that retrieve cryptographic keys for encrypting data via the KMIP protocol.


## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- MySQL Enterprise installed on docker (to simulate an external MySQL that requires TDE encryption.  i.e. encryption at rest.)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

Note: You will need Vault to be installed with an ADP KMIP license add-on.  Also note that a Premium license is required if you wish the KMIP listener to scale on the Vault cluster.  For Standard or Plus license, the KMIP listener is only on the leader node.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

MySQL Enterprise Edition is required to use keyring_okv KMIP plugin.
- https://dev.mysql.com/doc/refman/8.0/en/keyring-okv-plugin.html

We will require an Oracle account to access the MySQL Enterprise trial.

- Go to https://edelivery.oracle.com and create an account (Click on "Create Account").  Note: Register with a valid email and details otherwise the account might not be allowed to download.  Enter your account details below.
- Go to https://container-registry.oracle.com and login.
- Search for "mysql enterprise" and click on the MySQL Server Enterprise Edition repository link.
- Accept the License Agreement for MySQL Enterprise.
- See the tags of the container version that you wish to use and update the docker commands below.

In [ ]:
# Important! Update your oracle account details here.
export ORACLE_REG_EMAIL=
export ORACLE_REG_PASSWORD=

# Use the MySQL Enterprise docker image for linux/arm64
export MYSQL_TAG=8.3.0-aarch64
# This is for linux/amd64 platforms
#export MYSQL_TAG=8.3.0
#export MYSQL_TAG=latest

# Log into the Oracle docker registry
echo "$ORACLE_REG_PASSWORD" | docker login container-registry.oracle.com -u $ORACLE_REG_EMAIL --password-stdin
# Pull the MySQL Enterprise docker image
docker pull container-registry.oracle.com/mysql/enterprise-server:$MYSQL_TAG

# Setting up HashiCorp Vault

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
# Expose both the Vault API and the KMIP ports to the host machine.
export VAULT_PORT=8200
export VAULT_KMIP_PORT=5696
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_KMIP_PORT}:${VAULT_KMIP_PORT} \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# Vault KMIP Secrets Engine
This section demonstrates the KMIP secrets engine and how it functions as a KMIP server with MySQL Server TDE.

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/kmip

## Step 1 - Enable Vault KMIP Engine

In [ ]:
# Enable KMIP secrets engine at the default mount path
#vault secrets disable kmip
vault secrets enable kmip

In [ ]:
# Start KMIP Server and set client TLS certificate TTL to 365 days

# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT default_tls_client_ttl=365d
# Option 2 - RSA
#vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT tls_ca_key_type="rsa" tls_ca_key_bits=2048 default_tls_client_ttl=365d

# Check the KMIP configuration
echo
vault read kmip/config

## Step 2 - Creating the KMIP certificates for MySQL Server TDE

In [ ]:
# Create a folder for the KMIP certificates to be used by MySQL Server
mkdir -p root/mysql-keyring-okv/ssl

In [ ]:
# Read and store the CA certificate.  Note that in linux, the casing of the name needs to be CA.pem and not ca.pem.
# On MacOS, the casing doesn't matter.
vault read -format=json kmip/ca | jq -r .data.ca_pem > root/mysql-keyring-okv/ssl/CA.pem

echo "CA certification:"
cat root/mysql-keyring-okv/ssl/CA.pem

## Step 2a - Create KMIP Scope in Vault

The KMIP secrets engine uses the concept of scopes to partition KMIP managed object storage into multiple named buckets. Within a scope, roles can be created which dictate the set of allowed operations that the particular role can perform. TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In order to generate client certificates for KMIP clients to interact with Vault's KMIP server, we must first create a scope and role and specify the desired set of allowed operations for it.

In [ ]:
# Settings for the KMIP scope and role to be created
export KMIP_SVC_NAME=my-service
export KMIP_ROLE_NAME=admin
# Create a scope
vault write -f kmip/scope/$KMIP_SVC_NAME

# List scopes and verify the KMIP scope has been created
echo
vault list kmip/scope

In [ ]:
# Create the KMIP role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME operation_all=true tls_client_ttl=365d

# List roles and verify the KMIP role has been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

In [ ]:
# Optional - View the role details
vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME

In [ ]:
# Generate the required certificates
vault write -format=json \
  kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME/credential/generate \
  format=pem > credential.json

# Save the certificate and private key
jq -r .data.certificate < credential.json > root/mysql-keyring-okv/ssl/cert.pem
jq -r .data.private_key < credential.json > root/mysql-keyring-okv/ssl/key.pem

# Remove the temp file
rm credential.json

## Step 3 - Configuring MySQL Server for TDE with Vault as the KMIP server

In [ ]:
# Get the docker IP address of the Vault server
export VAULT_IP=$(docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' vault-enterprise)
echo "Vault IP Address is: $VAULT_IP"
echo "Vault KMIP Port is: $VAULT_KMIP_PORT"

# Create the okvclient.ora file.  Specifies the KMIP server for MySQL Server.
# For this demo, we will be using the same Vault server as the standby.
# Note: Do not create this file on a windows machines as it inserts a CRLF character instead of a LF character on the line break.
# This will cause an error on MySQL server on load.
echo
tee root/mysql-keyring-okv/okvclient.ora <<EOF
SERVER=$VAULT_IP:$VAULT_KMIP_PORT
STANDBY_SERVER=$VAULT_IP:$VAULT_KMIP_PORT
EOF

In [ ]:
# Configure my.cnf that MySQL Server will use.
tee root/my.cnf <<EOF
[mysqld]
port=3306
early-plugin-load=keyring_okv.so
keyring_okv_conf_dir=/mysql-keyring-okv
EOF

In [ ]:
# Run MySQL Enterprise in docker.
# Expose both the MySQL port to the host machine.
export MYSQL_PORT=3306
export MYSQL_TAG=8.3.0-aarch64

echo "MySQL Container Version Used: $MYSQL_TAG"
echo
docker run -d --rm --name mysql-enterprise \
--mount type=bind,src=./root/my.cnf,dst=/etc/my.cnf \
--mount type=bind,src=./root/mysql-keyring-okv,dst=/mysql-keyring-okv \
-p ${MYSQL_PORT}:${MYSQL_PORT} container-registry.oracle.com/mysql/enterprise-server:$MYSQL_TAG


In [ ]:
# Verify that the MySQL Server docker container is running
docker ps

## Step 4 - MySQL Server TDE Configuration Steps

In [ ]:
# set mysql alias to make it easier to execute mysql commands to the mysql docker container
alias mysql="docker exec -it mysql-enterprise mysql"

In [ ]:
# Get the generated root password
export MYSQL_GEN_ROOT_PASSWORD=$(docker logs mysql-enterprise 2>&1 | grep GENERATED | cut -d " " -f5)
echo "Generated MySQL root password: $MYSQL_GEN_ROOT_PASSWORD"

In [ ]:
# You will need to change the generated root password before running any other commands
export MYSQL_ROOT_PASSWORD=root
mysql --connect-expired-password -u root -p"$MYSQL_GEN_ROOT_PASSWORD" -e "ALTER USER 'root'@'localhost' IDENTIFIED BY '$MYSQL_ROOT_PASSWORD';"

In [ ]:
# Verify that the keyring_okv plugin is installed and the status is ACTIVE
mysql -u root -p'root' -e \
"SELECT PLUGIN_NAME, PLUGIN_STATUS
FROM INFORMATION_SCHEMA.PLUGINS
WHERE PLUGIN_NAME LIKE 'keyring%';"

In [ ]:
# Install the general purpose keyring key-management functions
# https://dev.mysql.com/doc/refman/8.0/en/keyring-functions-general-purpose.html
mysql -u root -p'root' -e \
"INSTALL PLUGIN keyring_udf SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_generate RETURNS INTEGER
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_fetch RETURNS STRING
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_length_fetch RETURNS INTEGER
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_type_fetch RETURNS STRING
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_store RETURNS INTEGER
SONAME 'keyring_udf.so';
CREATE FUNCTION keyring_key_remove RETURNS INTEGER
SONAME 'keyring_udf.so';"

## Step 5 - Testing MySQL TDE functionality

In [ ]:
# Create a new random key and store it on the keyring
# If the KMIP configuration is not done right, this function will return an error.
# Also note it will throw an error if the MyKey1 already exists
mysql -u root -p'root' -e \
"SELECT keyring_key_generate('MyKey1', 'AES', 32);"

In [ ]:
# Test creating a test database and table
mysql -u root -p'root' -e \
"CREATE DATABASE scripting;"

mysql -u root -p'root' -e \
"USE scripting;
CREATE TABLE test_encryption (
  ID int(10) unsigned NOT NULL AUTO_INCREMENT,
  server_name varchar(15) NOT NULL,
  PRIMARY KEY (id)
) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=latin1 ENCRYPTION = 'Y';"


In [ ]:
# Run this command to insert test data into the table
mysql -u root -p'root' -e \
"USE scripting;
INSERT INTO test_encryption(server_name)
VALUES (CONCAT('websvr-', SUBSTR(MD5(RAND()),1,8)));"

# View the records in the table
mysql -u root -p'root' -e \
"USE scripting;
SELECT * FROM test_encryption;"

In [ ]:
# Test master key rotation
mysql -u root -p'root' -e \
"ALTER INSTANCE ROTATE INNODB MASTER KEY;"

In [ ]:
# Simulate that the KMIP server is down by changing the listening port to 5697
vault write kmip/config listen_addrs=0.0.0.0:5697

In [ ]:
# Verify that I can still insert and select data
mysql -u root -p'root' -e \
"USE scripting;
INSERT INTO test_encryption(server_name)
VALUES (CONCAT('websvr-', SUBSTR(MD5(RAND()),1,8)));"

# View the records in the table
mysql -u root -p'root' -e \
"USE scripting;
SELECT * FROM test_encryption;"

In [ ]:
# Try rotating the master key.  As the KMIP server is "down", you should get an error.
mysql -u root -p'root' -e \
"ALTER INSTANCE ROTATE INNODB MASTER KEY;"

In [ ]:
# Simulate restoring back the KMIP server by updating back to the original KMIP port number
vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT

In [ ]:
# Try rotating the master key again.  As the KMIP server is now "restored", this command should now return no errors.
mysql -u root -p'root' -e \
"ALTER INSTANCE ROTATE INNODB MASTER KEY;"

# Cleanup

In [ ]:
# Cleanup

# Disable KMIP secrets engine
vault secrets disable kmip

# Remove KMIP certificate demo files
rm -rf root

# Stop Vault container
docker stop vault-enterprise

# Stop MySQL container
docker stop mysql-enterprise